# Efficiently calculating inventory parameters

[Accompanying blog post](https://chris.mutel.org/calculating-inventory-flows.html)

In [1]:
import brightway2 as bw

## Set up a new project

In [2]:
bw.projects.set_current("Inventory flows")

In [3]:
bw.create_default_biosphere3()
bw.create_core_migrations()

Creating default biosphere



Writing activities to SQLite3 database:
0%                          100%
[###                           ] | ETA: 00:00:01

Applying strategy: normalize_units
Applying strategy: drop_unspecified_subcategories
Applied 2 strategies in 0.02 seconds


[##############################] | ETA: 00:00:00
Total time elapsed: 00:00:01


Title: Writing activities to SQLite3 database:
  Started: 11/13/2018 08:31:10
  Finished: 11/13/2018 08:31:11
  Total time elapsed: 00:00:01
  CPU %: 63.70
  Memory %: 0.87
Created database: biosphere3
Creating default LCIA methods

Applying strategy: normalize_units
Applying strategy: set_biosphere_type
Applying strategy: drop_unspecified_subcategories
Applying strategy: link_iterable_by_fields
Applied 4 strategies in 2.03 seconds
Wrote 850 LCIA methods with 220699 characterization factors
Creating core data migrations



In [4]:
ei = bw.SingleOutputEcospold2Importer(
    "/Users/cmutel/Documents/LCA/Ecoinvent/3.5/cutoff/datasets", 
    "ecoinvent 3.5 cutoff"
)
ei.apply_strategies()
ei.statistics()

Extracting XML data from 16022 datasets
Extracted 16022 datasets in 83.12 seconds
Applying strategy: normalize_units
Applying strategy: update_ecoinvent_locations
Applying strategy: remove_zero_amount_coproducts
Applying strategy: remove_zero_amount_inputs_with_no_activity
Applying strategy: remove_unnamed_parameters
Applying strategy: es2_assign_only_product_with_amount_as_reference_product
Applying strategy: assign_single_product_as_activity
Applying strategy: create_composite_code
Applying strategy: drop_unspecified_subcategories
Applying strategy: fix_ecoinvent_flows_pre35
Applying strategy: drop_temporary_outdated_biosphere_flows
Applying strategy: link_biosphere_by_flow_uuid
Applying strategy: link_internal_technosphere_by_composite_code
Applying strategy: delete_exchanges_missing_activity
Applying strategy: delete_ghost_exchanges
Applying strategy: remove_uncertainty_from_negative_loss_exchanges
Applying strategy: fix_unreasonably_high_lognormal_uncertainties
Applying strategy: 

(16022, 544735, 0)

In [5]:
ei.write_database()

Writing activities to SQLite3 database:
0%                          100%
[##############################] | ETA: 00:00:00
Total time elapsed: 00:00:54


Title: Writing activities to SQLite3 database:
  Started: 11/13/2018 08:45:22
  Finished: 11/13/2018 08:46:17
  Total time elapsed: 00:00:54
  CPU %: 89.10
  Memory %: 8.24
Created database: ecoinvent 3.5 cutoff


Brightway2 SQLiteBackend: ecoinvent 3.5 cutoff

## Calculating flows via the supply array

In [4]:
car = next(x for x in bw.Database("ecoinvent 3.5 cutoff") 
           if x['name'] == "passenger car production, petrol/natural gas")
car

'passenger car production, petrol/natural gas' (kilogram, GLO, None)

In [4]:
lca = bw.LCA({car: 1000})  # Assume car is 1000 kilograms
lca.lci()

In [5]:
STEEL_NAMES = {
    'steel production, converter, chromium steel 18/8',
    'steel production, converter, low-alloyed',
    'steel production, converter, unalloyed',
    'steel production, electric, chromium steel 18/8',
    'steel production, electric, low-alloyed',
    # Not actually production processes
    #     'steel production, chromium steel 18/8, hot rolled',
    #     'steel production, low-alloyed, hot rolled',
    #     'reinforcing steel production',
}
STEELS = [x for x in bw.Database("ecoinvent 3.5 cutoff") 
          if x['name'] in STEEL_NAMES]

In [6]:
sum(lca.supply_array[lca.activity_dict[act.key]] for act in STEELS)

999.7328308178185

## New biosphere flows and LCIA method

In [34]:
bw.Database("Inventory flows").write({
    ('Inventory flows', 'steel'): {
        'unit': 'kilogram',
        'type': 'inventory flow',
        'categories': ('inventory',),
    }
})

Writing activities to SQLite3 database:
0%  100%
[#] | ETA: 00:00:00
Total time elapsed: 00:00:00


Title: Writing activities to SQLite3 database:
  Started: 11/13/2018 09:39:55
  Finished: 11/13/2018 09:39:55
  Total time elapsed: 00:00:00
  CPU %: 106.80
  Memory %: 3.38


In [43]:
for act in STEELS:
    act.new_exchange(**{
        'input': ('Inventory flows', 'steel'),
        'type': 'biosphere',
        'amount': 1,  # Assumes production amount of 1
    }).save() 

In [40]:
m = bw.Method(("Inventory flows", "Steel"))
m.register(unit='kilogram')
m.write([
    (('Inventory flows', 'steel'), 1)
])

In [62]:
bw.Database("ecoinvent 3.5 cutoff").process()

In [27]:
lca = bw.LCA({car: 1000}, ("Inventory flows", "Steel"))
lca.lci()
lca.lcia()
lca.score

999.7328308178185

In [105]:
def without_double_counting(lca, activity_of_interest, activities_to_exclude):
    """Calculate a new LCIA score for ``activity_of_interest`` but excluding 
    contributions from ``activities_to_exclude``.
    
    * ``lca`` is an ``LCA`` object for which LCI and LCIA have already been calculated
    * ``activity_of_interest`` is a demand dictionary, e.g. {some_activity: amount}
    * ``activities_to_exclude`` is an iterable of activity objects or keys
    
    Returns the LCIA score.
    """
    assert hasattr(lca, "characterized_inventory"), "Must do LCI and LCIA first"
    
    tm_original = lca.technosphere_matrix.copy()
    
    to_key = lambda x: x if isinstance(x, tuple) else x.key
    
    exclude = set([to_key(o) for o in activities_to_exclude]).difference(
              set([to_key(o) for o in activity_of_interest]))
    
    for activity in exclude:
        row = lca.product_dict[activity]
        col = lca.activity_dict[activity]
        production_amount = lca.technosphere_matrix[row, col]
        lca.technosphere_matrix[row, :] *= 0
        lca.technosphere_matrix[row, col] = production_amount
        
    lca.redo_lcia(activity_of_interest)    
    lca.technosphere_matrix = tm_original
    return lca.score

In [24]:
# market for glider, passenger car
glider = ('ecoinvent 3.5 cutoff', '3190a5aaecaaa169947d055586a0a4ae')

# market for internal combustion engine, passenger car
engine = ('ecoinvent 3.5 cutoff', 'e4bdb0c9a5612e4df90ac8c8cbc9692f')

In [29]:
for name, act in [("Glider", glider), ("Engine", engine)]:
    print(name, lca.supply_array[lca.activity_dict[act]])

Glider 739.8802761540312
Engine 260.1336051832947


In [106]:
without_double_counting(lca, {car: 1000}, [glider, engine])

0.8322468900755811

In [30]:
without_double_counting(lca, {glider: 739.8802761540312}, [car, engine])

840.408990053158

In [31]:
without_double_counting(lca, {engine: 260.1336051832947}, [car, glider])

158.49646168629997

In [32]:
(158.49646168629997 + 840.408990053158 + 0.8322468900755811) / 999.7328308178185

1.000004869112592

## How can 1 kg of glider induce production of more than 1 kg of steel?

In [60]:
lca = bw.LCA({glider: 1}, ("Inventory flows", "Steel"))
lca.lci()
lca.lcia()
lca.score

1.1358793666869558

In [75]:
def recursive_search(lca, fu, amount, total_score=None, level=0, max_level=3, cutoff=0.005, tab="\t"):        
    if level >= max_level:
        return
    
    lca.redo_lcia({fu: amount})
    if total_score is None:
        total_score = lca.score
    if abs(lca.score) < abs(total_score * cutoff):
        return
    print("{}{:g} | {:f} | {:g} | {}".format(tab * level, amount, lca.score / total_score, lca.score, fu))
    for exc in fu.technosphere():
        recursive_search(lca, exc.input, exc['amount'] * amount, 
                         total_score, level + 1, max_level, cutoff, tab)

In [55]:
recursive_search(lca, bw.get_activity(glider), 1, max_level=7, tab="> ")

1 | 1.000000 | 1.13588 | 'market for glider, passenger car' (kilogram, GLO, None)
> 1 | 1.000000 | 1.13588 | 'glider production, passenger car' (kilogram, GLO, None)
> > 0.163566 | 0.152812 | 0.173576 | 'market for steel, low-alloyed, hot rolled' (kilogram, GLO, None)
> > > 0.0271845 | 0.025396 | 0.0288464 | 'steel production, low-alloyed, hot rolled' (kilogram, RER, None)
> > > > 0.0271845 | 0.024172 | 0.0274568 | 'market for steel, low-alloyed' (kilogram, GLO, None)
> > > > > 0.0130224 | 0.011604 | 0.0131808 | 'steel production, converter, low-alloyed' (kilogram, RoW, None)
> > > > > 0.00858122 | 0.007597 | 0.00862873 | 'steel production, electric, low-alloyed' (kilogram, RoW, None)
> > > 0.136382 | 0.127417 | 0.14473 | 'steel production, low-alloyed, hot rolled' (kilogram, RoW, None)
> > > > 0.136382 | 0.006147 | 0.00698243 | 'market for hot rolling, steel' (kilogram, GLO, None)
> > > > > 0.119277 | 0.005377 | 0.00610805 | 'hot rolling, steel' (kilogram, RoW, None)
> > > > > > 0.005

### Which activities have the highest losses?

In [66]:
def recursive_search_2(lca, fu, amount, total_score=None, level=0, max_level=3, 
                     cutoff=0.005, tab="\t", seen=set()):        
    if level >= max_level:
        return
    
    seen.add(fu)
    lca.redo_lcia({fu: amount})
    if total_score is None:
        total_score = lca.score
    if abs(lca.score) < abs(total_score * cutoff):
        return
    for exc in fu.technosphere():
        recursive_search_2(lca, exc.input, exc['amount'] * amount, 
                         total_score, level + 1, max_level, cutoff, tab, seen)
        
    return seen

In [67]:
seen = recursive_search_2(lca, bw.get_activity(glider), 1, max_level=7, tab="> ")

In [63]:
results = []

for o in seen:
    lca.redo_lcia({o: 1})
    results.append((lca.score, o))
    
sorted(results, reverse=True)

[(34607882.40763581,
  'market for blast oxygen furnace converter' (unit, GLO, None)),
 (34598681.610800944,
  'blast oxygen furnace converter production' (unit, RER, None)),
 (18838386.267116528,
  'market for electric arc furnace converter' (unit, GLO, None)),
 (18835290.41979831,
  'electric arc furnace converter construction' (unit, RER, None)),
 (15348629.08600751, 'market for road vehicle factory' (unit, GLO, None)),
 (61541.81016853929, 'market for rolling mill' (unit, GLO, None)),
 (2.3530684581297403, 'market for molybdenum trioxide' (kilogram, GLO, None)),
 (2.143629007176738, 'market for light emitting diode' (kilogram, GLO, None)),
 (2.1141140059582977, 'market for molybdenite' (kilogram, GLO, None)),
 (1.7265083785749016,
  'market for printed wiring board, mounted mainboard, desktop computer, Pb free' (kilogram, GLO, None)),
 (1.5875346673014703,
  'market for sawnwood, softwood, raw, dried (u=20%)' (cubic meter, RER, None)),
 (1.58753466730147,
  'market for sawnwood, so

### What is going on with soft wood production?

In [72]:
# 'market for sawnwood, softwood, raw, dried (u=20%)' (cubic meter, RER, None)
wood = ('ecoinvent 3.5 cutoff', 'c028ce5c9f43e7f4fd03763f4e71f33c')

# 'market for sawnwood, board, softwood, raw, dried (u=20%)' (cubic meter, GLO, None)
wood = ('ecoinvent 3.5 cutoff', '690c108e2ffdad3444d28625b8dcddd3')

In [71]:
[(o, o.key) for o in bw.Database("ecoinvent 3.5 cutoff") 
 if o['name'] == 'market for sawnwood, board, softwood, raw, dried (u=20%)']

[('market for sawnwood, board, softwood, raw, dried (u=20%)' (cubic meter, GLO, None),
  ('ecoinvent 3.5 cutoff', '690c108e2ffdad3444d28625b8dcddd3'))]

In [76]:
recursive_search(lca, bw.get_activity(wood), 1, max_level=7, tab="> ", cutoff=0.01)

1 | 1.000000 | 1.58714 | 'market for sawnwood, board, softwood, raw, dried (u=20%)' (cubic meter, GLO, None)
> 93.8108 | 0.094711 | 0.15032 | 'market group for transport, freight, lorry, unspecified' (ton kilometer, GLO, None)
> > 27.1113 | 0.027472 | 0.0436025 | 'market for transport, freight, lorry, unspecified' (ton kilometer, RER, None)
> > > 12.1212 | 0.012689 | 0.020139 | 'transport, freight, lorry, all sizes, EURO3 to generic market for transport, freight, lorry, unspecified' (ton kilometer, RER, None)
> > > 10.5785 | 0.010456 | 0.0165952 | 'transport, freight, lorry, all sizes, EURO4 to generic market for transport, freight, lorry, unspecified' (ton kilometer, RER, None)
> > 66.6995 | 0.067239 | 0.106717 | 'market for transport, freight, lorry, unspecified' (ton kilometer, RoW, None)
> > > 29.8206 | 0.031056 | 0.0492895 | 'transport, freight, lorry, all sizes, EURO3 to generic market for transport, freight, lorry, unspecified' (ton kilometer, RoW, None)
> > > > 10.0552 | 0.0121

> > > > > > 0.0797677 | 0.068179 | 0.108209 | 'harvesting, forestry harvester' (hour, RoW, None)
> > > > 2.61675 | 0.065804 | 0.104439 | 'market for transport, freight, light commercial vehicle' (ton kilometer, RoW, None)
> > > > > 2.61675 | 0.065804 | 0.104439 | 'transport, freight, light commercial vehicle' (ton kilometer, RoW, None)
> > > > > > 6.26739e-05 | 0.052655 | 0.0835703 | 'market for light commercial vehicle' (unit, GLO, None)
> > > > 22.719 | 0.037775 | 0.0599538 | 'market for transport, freight train' (ton kilometer, RoW, None)
> > > > > 7.95838 | 0.013292 | 0.0210963 | 'transport, freight train, electricity' (ton kilometer, RoW, None)
> > > > > 14.7606 | 0.024483 | 0.0388575 | 'transport, freight train, diesel' (ton kilometer, RoW, None)
> > > > > > 0.00137273 | 0.013743 | 0.0218119 | 'market for railway track' (meter-year, GLO, None)
> > > 8.40814 | 0.013724 | 0.0217811 | 'market group for electricity, medium voltage' (kilowatt hour, RAS, None)


No clear answer - a lot of steel in the saw mill construction, and then small amounts in the different forestry equipments and transportation steps. Needs to be investigated further, as this result feels wrong.

# Vehicles sector: Exclude LDV from HDV

LDV (light duty vehicles) have a gross vehicle weight of up to 8 tons. In ecoinvent, LDVs are classified as "3.5-7.5 ton". HDV (heavy duty vehicles) are... heavier :)

In [85]:
trucks = next(x for x in bw.Database("ecoinvent 3.5 cutoff") 
           if x['name'] == "market group for transport, freight, lorry, unspecified")
trucks

'market group for transport, freight, lorry, unspecified' (ton kilometer, GLO, None)

In [88]:
ldvs = [o for o in bw.Database("ecoinvent 3.5 cutoff") 
        if o['name'].startswith("transport, freight, lorry") and "3.5-7.5 ton" in o['name']]

In [90]:
lca = bw.LCA({trucks: 1}, ("Inventory flows", "Steel"))
lca.lci()
lca.lcia()
lca.score

0.0016023705343866921

In [92]:
0.0016023705343866921 - without_double_counting(lca, {trucks: 1}, ldvs)

2.168404344971009e-18

As expected, in this particular case the marginal steel input is quite small, as LDVs only contribute 3% of ton-kilometers, and have proportionately small amounts of truck mass.

# Exogenous demand: Transport and electricity

Sometimes the amounts of materials demanded will come from external models, such as integrated assessment models. In this case, we don't need to look up demands in the supply array, but just supply them directly.

In cases where one aggregate demand from an external model maps to multiple ecoinvent activities, we allocate based on reported production volumes.

In [100]:
def get_production_volume(act):
    return next(iter(act.production()))['production volume']

In [103]:
all_transport = {
    o: get_production_volume(o) 
    for o in bw.Database("ecoinvent 3.5 cutoff") 
    if o['name'].startswith("transport, freight, lorry")
}
total = sum(all_transport.values())
all_transport = {k: v / total * 1e4 for k, v in all_transport.items()}

In [104]:
all_electricity = {
    o: get_production_volume(o) 
    for o in bw.Database("ecoinvent 3.5 cutoff") 
    if o['name'].startswith("market for electricity,")
}
total = sum(all_electricity.values())
all_electricity = {k: v / total * 1e2 for k, v in all_electricity.items()}

In [107]:
without_double_counting(lca, all_transport, all_electricity)

0.001582994381059183

In [110]:
lca.redo_lcia(all_transport)
lca.score

0.0016145330490756776

In [108]:
without_double_counting(lca, all_electricity, all_transport)

0.00216614294414289

In [111]:
lca.redo_lcia(all_electricity)
lca.score

0.0021898578697670595